# 02. 탐색적 데이터 분석 (EDA)

Phase 2: EDA 및 Feature Engineering

## 목표
- 2.1 갭 상승 기초 통계 분석 (연도별/월별 빈도, 갭 크기 분포)
- 2.2 타겟 분포 분석 (전체 승률, 갭 크기별 승률)
- 2.3 Feature 상관관계 및 Feature Importance 예비 분석

## 1. 라이브러리 임포트 및 데이터 로드

In [ ]:
import sys
from pathlib import Path

# 프로젝트 루트 경로 추가
project_root = Path().resolve().parent
sys.path.insert(0, str(project_root))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# 한글 폰트 설정 (선택사항)
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['axes.unicode_minus'] = False

# 그래프 스타일
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✓ 라이브러리 임포트 완료")

In [ ]:
df.to_csv("korea.csv")

In [ ]:
# 전처리된 데이터 로드
data_path = "../data/processed/preprocessed_df_full.parquet"

print(f"데이터 로드 중: {data_path}")
df = pd.read_parquet(data_path)

print(f"\n✓ 데이터 로드 완료")
print(f"  - Shape: {df.shape}")
print(f"  - 날짜 범위: {df['date'].min()} ~ {df['date'].max()}")
print(f"  - 종목 수: {df['InfoCode'].nunique():,}")
print(f"  - 메모리 사용량: {df.memory_usage(deep=True).sum() / 1024 / 1024:.2f} MB")

In [ ]:
# 데이터 기본 정보 확인
print("=" * 80)
print("데이터 기본 정보")
print("=" * 80)

print(f"\n전체 컬럼 ({len(df.columns)}개):")
for i, col in enumerate(df.columns, 1):
    print(f"  {i:2d}. {col:30s} - dtype: {df[col].dtype}")

print(f"\n데이터 샘플 (처음 5개):")
display(df.head())

## 2. 갭 상승 기초 통계 분석

### 2.1 전체 갭 상승 이벤트 통계

In [ ]:
# 갭 상승 이벤트 필터링
df_gap_up = df[df['is_gap_up'] == 1].copy()

print("=" * 80)
print("전체 갭 상승 이벤트 통계")
print("=" * 80)

print(f"\n전체 거래일 수: {len(df):,}")
print(f"갭 상승 이벤트 수: {len(df_gap_up):,}")
print(f"갭 상승 비율: {len(df_gap_up)/len(df)*100:.2f}%")

print(f"\n갭 상승률 분포:")
print(df_gap_up['gap_pct'].describe())

print(f"\n갭 크기 카테고리별 분포:")
gap_category_counts = df_gap_up['gap_size_category'].value_counts().sort_index()
for category, count in gap_category_counts.items():
    pct = count / len(df_gap_up) * 100
    print(f"  {category:10s}: {count:8,} ({pct:5.2f}%)")

In [ ]:
# 갭 상승률 분포 시각화
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# 히스토그램
axes[0].hist(df_gap_up['gap_pct'], bins=100, edgecolor='black', alpha=0.7)
axes[0].axvline(df_gap_up['gap_pct'].mean(), color='red', linestyle='--', 
                label=f"Mean: {df_gap_up['gap_pct'].mean():.2f}%")
axes[0].axvline(df_gap_up['gap_pct'].median(), color='green', linestyle='--', 
                label=f"Median: {df_gap_up['gap_pct'].median():.2f}%")
axes[0].set_xlabel('Gap Percentage (%)')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Gap Up Distribution (Histogram)')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Box plot (갭 크기 카테고리별)
df_gap_up.boxplot(column='gap_pct', by='gap_size_category', ax=axes[1])
axes[1].set_xlabel('Gap Size Category')
axes[1].set_ylabel('Gap Percentage (%)')
axes[1].set_title('Gap Distribution by Size Category')
plt.suptitle('')  # 기본 제목 제거

plt.tight_layout()
plt.show()

### 2.2 연도별 갭 발생 빈도

In [ ]:
# 연도별 갭 발생 빈도
yearly_gaps = df_gap_up.groupby('year').agg({
    'InfoCode': 'count',  # 갭 이벤트 수
    'gap_pct': ['mean', 'median'],
    'target_direction': 'mean'  # 승률
}).round(2)

yearly_gaps.columns = ['Gap_Events', 'Avg_Gap_Pct', 'Median_Gap_Pct', 'Win_Rate']
yearly_gaps['Win_Rate'] = (yearly_gaps['Win_Rate'] * 100).round(1)

print("=" * 80)
print("연도별 갭 상승 통계")
print("=" * 80)
print(yearly_gaps)

# 시각화
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 연도별 갭 이벤트 수
axes[0, 0].bar(yearly_gaps.index, yearly_gaps['Gap_Events'], alpha=0.7, edgecolor='black')
axes[0, 0].set_xlabel('Year')
axes[0, 0].set_ylabel('Number of Gap Events')
axes[0, 0].set_title('Gap Up Events by Year')
axes[0, 0].grid(True, alpha=0.3)

# 연도별 평균 갭 상승률
axes[0, 1].plot(yearly_gaps.index, yearly_gaps['Avg_Gap_Pct'], marker='o', linewidth=2)
axes[0, 1].set_xlabel('Year')
axes[0, 1].set_ylabel('Average Gap (%)')
axes[0, 1].set_title('Average Gap Percentage by Year')
axes[0, 1].grid(True, alpha=0.3)

# 연도별 승률
axes[1, 0].plot(yearly_gaps.index, yearly_gaps['Win_Rate'], marker='o', 
                color='green', linewidth=2)
axes[1, 0].axhline(50, color='red', linestyle='--', label='50% baseline')
axes[1, 0].set_xlabel('Year')
axes[1, 0].set_ylabel('Win Rate (%)')
axes[1, 0].set_title('Win Rate by Year')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# 연도별 Median Gap
axes[1, 1].plot(yearly_gaps.index, yearly_gaps['Median_Gap_Pct'], marker='o', 
                color='orange', linewidth=2)
axes[1, 1].set_xlabel('Year')
axes[1, 1].set_ylabel('Median Gap (%)')
axes[1, 1].set_title('Median Gap Percentage by Year')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### 2.3 월별 갭 발생 빈도

In [ ]:
# 월별 갭 발생 빈도
monthly_gaps = df_gap_up.groupby('month').agg({
    'InfoCode': 'count',
    'gap_pct': ['mean', 'median'],
    'target_direction': 'mean'
}).round(2)

monthly_gaps.columns = ['Gap_Events', 'Avg_Gap_Pct', 'Median_Gap_Pct', 'Win_Rate']
monthly_gaps['Win_Rate'] = (monthly_gaps['Win_Rate'] * 100).round(1)

print("=" * 80)
print("월별 갭 상승 통계")
print("=" * 80)
print(monthly_gaps)

# 시각화
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# 월별 갭 이벤트 수
axes[0].bar(monthly_gaps.index, monthly_gaps['Gap_Events'], alpha=0.7, edgecolor='black')
axes[0].set_xlabel('Month')
axes[0].set_ylabel('Number of Gap Events')
axes[0].set_title('Gap Up Events by Month')
axes[0].set_xticks(range(1, 13))
axes[0].grid(True, alpha=0.3)

# 월별 승률
axes[1].plot(monthly_gaps.index, monthly_gaps['Win_Rate'], marker='o', 
             color='green', linewidth=2)
axes[1].axhline(50, color='red', linestyle='--', label='50% baseline')
axes[1].set_xlabel('Month')
axes[1].set_ylabel('Win Rate (%)')
axes[1].set_title('Win Rate by Month')
axes[1].set_xticks(range(1, 13))
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### 2.4 요일별 갭 발생 빈도

In [ ]:
# 요일별 갭 발생 빈도
weekday_gaps = df_gap_up.groupby('day_of_week').agg({
    'InfoCode': 'count',
    'gap_pct': ['mean', 'median'],
    'target_direction': 'mean'
}).round(2)

weekday_gaps.columns = ['Gap_Events', 'Avg_Gap_Pct', 'Median_Gap_Pct', 'Win_Rate']
weekday_gaps['Win_Rate'] = (weekday_gaps['Win_Rate'] * 100).round(1)
weekday_gaps.index = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']

print("=" * 80)
print("요일별 갭 상승 통계")
print("=" * 80)
print(weekday_gaps)

# 시각화
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# 요일별 갭 이벤트 수
axes[0].bar(range(5), weekday_gaps['Gap_Events'], alpha=0.7, edgecolor='black')
axes[0].set_xlabel('Day of Week')
axes[0].set_ylabel('Number of Gap Events')
axes[0].set_title('Gap Up Events by Day of Week')
axes[0].set_xticks(range(5))
axes[0].set_xticklabels(weekday_gaps.index, rotation=45)
axes[0].grid(True, alpha=0.3)

# 요일별 승률
axes[1].plot(range(5), weekday_gaps['Win_Rate'], marker='o', 
             color='green', linewidth=2)
axes[1].axhline(50, color='red', linestyle='--', label='50% baseline')
axes[1].set_xlabel('Day of Week')
axes[1].set_ylabel('Win Rate (%)')
axes[1].set_title('Win Rate by Day of Week')
axes[1].set_xticks(range(5))
axes[1].set_xticklabels(weekday_gaps.index, rotation=45)
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 3. 타겟 분포 분석

### 3.1 전체 갭 상승 후 승률

In [ ]:
print("=" * 80)
print("전체 갭 상승 타겟 분포")
print("=" * 80)

# 전체 승률
win_rate = df_gap_up['target_direction'].mean() * 100
wins = df_gap_up['target_direction'].sum()
losses = len(df_gap_up) - wins

print(f"\n전체 갭 상승 이벤트: {len(df_gap_up):,}")
print(f"  - 상승 (종가 > 시가): {wins:,} ({win_rate:.2f}%)")
print(f"  - 하락 (종가 ≤ 시가): {losses:,} ({100-win_rate:.2f}%)")

# 수익률 통계
print(f"\n당일 수익률 통계:")
print(df_gap_up['target_return'].describe())

# 상승/하락별 수익률
df_up = df_gap_up[df_gap_up['target_direction'] == 1]
df_down = df_gap_up[df_gap_up['target_direction'] == 0]

print(f"\n상승 시 평균 수익률: {df_up['target_return'].mean():.2f}%")
print(f"하락 시 평균 손실률: {df_down['target_return'].mean():.2f}%")

# 기대 수익률
expected_return = df_gap_up['target_return'].mean()
print(f"\n기대 수익률: {expected_return:.2f}%")

In [ ]:
# 타겟 분포 시각화
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# 타겟 방향 분포 (파이 차트)
target_counts = df_gap_up['target_direction'].value_counts()
axes[0].pie(target_counts, labels=['Down (0)', 'Up (1)'], autopct='%1.1f%%', 
            startangle=90, colors=['lightcoral', 'lightgreen'])
axes[0].set_title(f'Target Direction Distribution\n(Win Rate: {win_rate:.1f}%)')

# 수익률 분포 (히스토그램)
axes[1].hist(df_gap_up['target_return'], bins=100, edgecolor='black', alpha=0.7)
axes[1].axvline(0, color='red', linestyle='--', linewidth=2, label='Break-even')
axes[1].axvline(expected_return, color='green', linestyle='--', linewidth=2, 
                label=f'Expected: {expected_return:.2f}%')
axes[1].set_xlabel('Intraday Return (%)')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Intraday Return Distribution')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# 상승/하락별 수익률 분포 (Box plot)
df_gap_up_with_label = df_gap_up.copy()
df_gap_up_with_label['direction_label'] = df_gap_up_with_label['target_direction'].map({0: 'Down', 1: 'Up'})
df_gap_up_with_label.boxplot(column='target_return', by='direction_label', ax=axes[2])
axes[2].set_xlabel('Target Direction')
axes[2].set_ylabel('Intraday Return (%)')
axes[2].set_title('Return Distribution by Direction')
plt.suptitle('')  # 기본 제목 제거

plt.tight_layout()
plt.show()

### 3.2 갭 크기별 승률

In [ ]:
# 갭 크기별 승률
gap_size_stats = df_gap_up.groupby('gap_size_category').agg({
    'InfoCode': 'count',
    'target_direction': 'mean',
    'target_return': ['mean', 'std'],
    'gap_pct': 'mean'
}).round(2)

gap_size_stats.columns = ['Count', 'Win_Rate', 'Avg_Return', 'Std_Return', 'Avg_Gap']
gap_size_stats['Win_Rate'] = (gap_size_stats['Win_Rate'] * 100).round(1)

print("=" * 80)
print("갭 크기별 성과 분석")
print("=" * 80)
print(gap_size_stats)

# 시각화
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# 갭 크기별 승률
gap_categories = gap_size_stats.index.astype(str)
axes[0].bar(range(len(gap_categories)), gap_size_stats['Win_Rate'], 
            alpha=0.7, edgecolor='black', color='green')
axes[0].axhline(50, color='red', linestyle='--', label='50% baseline')
axes[0].set_xlabel('Gap Size Category')
axes[0].set_ylabel('Win Rate (%)')
axes[0].set_title('Win Rate by Gap Size')
axes[0].set_xticks(range(len(gap_categories)))
axes[0].set_xticklabels(gap_categories)
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# 갭 크기별 평균 수익률
axes[1].bar(range(len(gap_categories)), gap_size_stats['Avg_Return'], 
            alpha=0.7, edgecolor='black', color='blue')
axes[1].axhline(0, color='red', linestyle='--', label='Break-even')
axes[1].set_xlabel('Gap Size Category')
axes[1].set_ylabel('Average Return (%)')
axes[1].set_title('Average Return by Gap Size')
axes[1].set_xticks(range(len(gap_categories)))
axes[1].set_xticklabels(gap_categories)
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### 3.3 갭 상승률 구간별 승률 (세분화)

In [ ]:
# 갭 상승률을 더 세분화된 구간으로 나누기
bins = [0, 1, 2, 3, 5, 10, 50]
labels = ['0-1%', '1-2%', '2-3%', '3-5%', '5-10%', '10%+']

df_gap_up['gap_bin'] = pd.cut(df_gap_up['gap_pct'], bins=bins, labels=labels, include_lowest=True)

gap_bin_stats = df_gap_up.groupby('gap_bin', observed=False).agg({
    'InfoCode': 'count',
    'target_direction': 'mean',
    'target_return': 'mean',
    'gap_pct': 'mean'
}).round(2)

gap_bin_stats.columns = ['Count', 'Win_Rate', 'Avg_Return', 'Avg_Gap']
gap_bin_stats['Win_Rate'] = (gap_bin_stats['Win_Rate'] * 100).round(1)

print("=" * 80)
print("갭 상승률 구간별 성과 분석")
print("=" * 80)
print(gap_bin_stats)

# 시각화
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# 구간별 이벤트 수와 승률
x_pos = range(len(gap_bin_stats))
axes[0].bar(x_pos, gap_bin_stats['Count'], alpha=0.5, label='Count', edgecolor='black')
ax0_twin = axes[0].twinx()
ax0_twin.plot(x_pos, gap_bin_stats['Win_Rate'], 'ro-', linewidth=2, markersize=8, label='Win Rate')
ax0_twin.axhline(50, color='red', linestyle='--', alpha=0.5)

axes[0].set_xlabel('Gap Percentage Range')
axes[0].set_ylabel('Count', color='blue')
ax0_twin.set_ylabel('Win Rate (%)', color='red')
axes[0].set_title('Count and Win Rate by Gap Range')
axes[0].set_xticks(x_pos)
axes[0].set_xticklabels(gap_bin_stats.index, rotation=45)
axes[0].legend(loc='upper left')
ax0_twin.legend(loc='upper right')
axes[0].grid(True, alpha=0.3)

# 구간별 평균 수익률
axes[1].bar(x_pos, gap_bin_stats['Avg_Return'], alpha=0.7, edgecolor='black')
axes[1].axhline(0, color='red', linestyle='--', label='Break-even')
axes[1].set_xlabel('Gap Percentage Range')
axes[1].set_ylabel('Average Return (%)')
axes[1].set_title('Average Return by Gap Range')
axes[1].set_xticks(x_pos)
axes[1].set_xticklabels(gap_bin_stats.index, rotation=45)
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### 3.4 섹터별 승률 (섹터 정보가 있는 경우)

In [ ]:
# 섹터 정보 확인
sector_coverage = df_gap_up['sector'].notna().sum() / len(df_gap_up) * 100

print(f"섹터 정보 커버리지: {sector_coverage:.1f}%")

if sector_coverage > 10:  # 10% 이상 커버되면 분석
    sector_stats = df_gap_up[df_gap_up['sector'].notna()].groupby('sector').agg({
        'InfoCode': 'count',
        'target_direction': 'mean',
        'target_return': 'mean',
        'gap_pct': 'mean'
    }).round(2)
    
    sector_stats.columns = ['Count', 'Win_Rate', 'Avg_Return', 'Avg_Gap']
    sector_stats['Win_Rate'] = (sector_stats['Win_Rate'] * 100).round(1)
    sector_stats = sector_stats.sort_values('Count', ascending=False)
    
    print("\n=" * 80)
    print("섹터별 성과 분석 (상위 10개)")
    print("=" * 80)
    print(sector_stats.head(10))
    
    # 시각화 (상위 10개 섹터)
    top_sectors = sector_stats.head(10)
    
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # 섹터별 승률
    axes[0].barh(range(len(top_sectors)), top_sectors['Win_Rate'], alpha=0.7, edgecolor='black')
    axes[0].axvline(50, color='red', linestyle='--', label='50% baseline')
    axes[0].set_xlabel('Win Rate (%)')
    axes[0].set_ylabel('Sector')
    axes[0].set_title('Win Rate by Sector (Top 10)')
    axes[0].set_yticks(range(len(top_sectors)))
    axes[0].set_yticklabels(top_sectors.index)
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # 섹터별 평균 수익률
    axes[1].barh(range(len(top_sectors)), top_sectors['Avg_Return'], alpha=0.7, edgecolor='black')
    axes[1].axvline(0, color='red', linestyle='--', label='Break-even')
    axes[1].set_xlabel('Average Return (%)')
    axes[1].set_ylabel('Sector')
    axes[1].set_title('Average Return by Sector (Top 10)')
    axes[1].set_yticks(range(len(top_sectors)))
    axes[1].set_yticklabels(top_sectors.index)
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
else:
    print("\n⚠ 섹터 정보가 부족하여 분석을 생략합니다.")

## 4. Feature 상관관계 분석

### 4.1 수치형 Feature 선택 및 상관관계 계산

In [ ]:
# 수치형 Feature 선택 (갭 상승 데이터만)
numeric_features = [
    # 갭 관련
    'gap_pct',
    
    # 전일 패턴
    'prev_return', 'prev_range_pct', 'prev_upper_shadow', 'prev_lower_shadow',
    
    # 거래량
    'volume_ratio',
    
    # 기술적 지표
    'rsi_14', 'atr_14', 'atr_ratio', 'bollinger_position',
    'return_5d', 'return_20d', 'consecutive_up_days',
    
    # 시장 컨텍스트
    'spy_gap_pct', 'qqq_gap_pct', 'market_gap_diff',
    
    # 타겟
    'target_direction', 'target_return'
]

# 실제로 존재하는 컬럼만 선택
available_features = [f for f in numeric_features if f in df_gap_up.columns]

print(f"분석 가능한 수치형 Features: {len(available_features)}개")
print(f"\nFeature 리스트:")
for i, feat in enumerate(available_features, 1):
    print(f"  {i:2d}. {feat}")

In [ ]:
# 상관관계 계산
correlation_matrix = df_gap_up[available_features].corr()

# 타겟과의 상관관계 (target_direction)
target_corr = correlation_matrix['target_direction'].sort_values(ascending=False)

print("=" * 80)
print("타겟(target_direction)과의 상관관계")
print("=" * 80)
print(target_corr)

# 상위/하위 Feature 표시
print(f"\n상위 5개 (양의 상관관계):")
print(target_corr.head(6)[1:])  # target_direction 자신 제외

print(f"\n하위 5개 (음의 상관관계):")
print(target_corr.tail(5))

In [ ]:
# 상관관계 히트맵 시각화
plt.figure(figsize=(16, 14))
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
            center=0, square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Feature Correlation Matrix', fontsize=16, pad=20)
plt.tight_layout()
plt.show()

In [ ]:
# 타겟과의 상관관계 바 차트
target_corr_plot = target_corr.drop('target_direction')  # 자기 자신 제외

plt.figure(figsize=(12, 8))
colors = ['green' if x > 0 else 'red' for x in target_corr_plot.values]
plt.barh(range(len(target_corr_plot)), target_corr_plot.values, color=colors, alpha=0.7, edgecolor='black')
plt.yticks(range(len(target_corr_plot)), target_corr_plot.index)
plt.xlabel('Correlation with target_direction')
plt.title('Feature Correlation with Target Direction', fontsize=14)
plt.axvline(0, color='black', linestyle='--', linewidth=1)
plt.grid(True, alpha=0.3, axis='x')
plt.tight_layout()
plt.show()

### 4.2 Feature 간 상관관계 (다중공선성 확인)

In [ ]:
# Feature 간 높은 상관관계 찾기 (절댓값 > 0.7)
print("=" * 80)
print("Feature 간 높은 상관관계 (|r| > 0.7)")
print("=" * 80)

high_corr_pairs = []
for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        if abs(correlation_matrix.iloc[i, j]) > 0.7:
            high_corr_pairs.append((
                correlation_matrix.columns[i],
                correlation_matrix.columns[j],
                correlation_matrix.iloc[i, j]
            ))

if high_corr_pairs:
    high_corr_df = pd.DataFrame(high_corr_pairs, columns=['Feature_1', 'Feature_2', 'Correlation'])
    high_corr_df = high_corr_df.sort_values('Correlation', ascending=False, key=abs)
    print(high_corr_df)
    print(f"\n⚠ 다중공선성 가능성: {len(high_corr_pairs)}개 쌍 발견")
else:
    print("\n✓ 높은 상관관계를 가진 Feature 쌍이 없습니다.")

### 4.3 주요 Feature 산점도 (타겟과의 관계)

In [ ]:
# 타겟과 상관관계가 높은 상위 6개 Feature 선택
top_features = target_corr.drop('target_direction').head(6).index.tolist()

# 산점도 그리기
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.ravel()

for idx, feature in enumerate(top_features):
    # 샘플링 (너무 많으면 느림)
    sample_size = min(10000, len(df_gap_up))
    df_sample = df_gap_up.sample(n=sample_size, random_state=42)
    
    # 타겟별로 색상 구분
    up_mask = df_sample['target_direction'] == 1
    down_mask = df_sample['target_direction'] == 0
    
    axes[idx].scatter(df_sample[down_mask][feature], df_sample[down_mask]['target_return'], 
                      alpha=0.3, s=10, c='red', label='Down')
    axes[idx].scatter(df_sample[up_mask][feature], df_sample[up_mask]['target_return'], 
                      alpha=0.3, s=10, c='green', label='Up')
    
    axes[idx].axhline(0, color='black', linestyle='--', linewidth=1, alpha=0.5)
    axes[idx].set_xlabel(feature)
    axes[idx].set_ylabel('target_return')
    axes[idx].set_title(f'{feature} vs target_return\n(corr: {target_corr[feature]:.3f})')
    axes[idx].legend()
    axes[idx].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5. Feature Importance 예비 분석

### 5.1 간단한 Tree 모델로 Feature Importance 확인

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, precision_recall_curve

# NaN 제거 및 학습 데이터 준비
feature_cols = [f for f in available_features if f not in ['target_direction', 'target_return']]
df_model = df_gap_up[feature_cols + ['target_direction']].dropna()

print(f"학습 데이터 Shape: {df_model.shape}")
print(f"Feature 개수: {len(feature_cols)}")

if len(df_model) > 100:  # 최소 데이터 확인
    X = df_model[feature_cols]
    y = df_model['target_direction']
    
    # 클래스 분포 확인
    print(f"\n클래스 분포:")
    print(f"  하락(0): {(y==0).sum():,} ({(y==0).sum()/len(y)*100:.1f}%)")
    print(f"  상승(1): {(y==1).sum():,} ({(y==1).sum()/len(y)*100:.1f}%)")
    
    # Train/Test 분할
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )
    
    print(f"\nTrain set: {len(X_train):,}")
    print(f"Test set: {len(X_test):,}")
    
    # Random Forest 학습 (개선된 하이퍼파라미터)
    print("\n=== Random Forest 학습 (개선 버전) ===")
    print("하이퍼파라미터:")
    print("  - n_estimators: 300 (100 → 300)")
    print("  - max_depth: 15 (10 → 15)")
    print("  - min_samples_split: 10")
    print("  - class_weight: 'balanced'")
    print("  - Optimal Threshold: 0.4 (분석 결과 반영)")
    
    rf_model = RandomForestClassifier(
        n_estimators=300,           # 트리 개수 증가
        max_depth=15,                # 깊이 증가
        min_samples_split=10,        # 분할 최소 샘플 수
        class_weight='balanced',     # 클래스 불균형 처리
        random_state=42,
        n_jobs=-1
    )
    rf_model.fit(X_train, y_train)
    
    # 성능 평가
    train_acc = rf_model.score(X_train, y_train)
    test_acc = rf_model.score(X_test, y_test)
    
    print(f"\n✓ 학습 완료")
    print(f"  Train Accuracy: {train_acc:.3f}")
    print(f"  Test Accuracy: {test_acc:.3f}")
    
    # 예측 (Threshold 0.5 기본)
    y_pred_default = rf_model.predict(X_test)
    y_proba = rf_model.predict_proba(X_test)[:, 1]  # 상승 확률
    
    # 예측 (Threshold 0.4 최적)
    y_pred_optimal = (y_proba >= 0.4).astype(int)
    
    # === Threshold 0.5 (기본) 결과 ===
    print("\n" + "=" * 80)
    print("📊 Threshold 0.5 (기본값) 결과")
    print("=" * 80)
    cm_default = confusion_matrix(y_test, y_pred_default)
    print("\n혼동 행렬:")
    print(cm_default)
    print(f"\n실제 하락 중 하락 예측: {cm_default[0,0]:,} / {cm_default[0].sum():,} ({cm_default[0,0]/cm_default[0].sum()*100:.1f}%)")
    print(f"실제 상승 중 상승 예측: {cm_default[1,1]:,} / {cm_default[1].sum():,} ({cm_default[1,1]/cm_default[1].sum()*100:.1f}%)")
    
    print("\n분류 성능:")
    print(classification_report(y_test, y_pred_default, target_names=['하락(0)', '상승(1)']))
    
    # === Threshold 0.4 (최적) 결과 ===
    print("\n" + "=" * 80)
    print("⭐ Threshold 0.4 (최적값) 결과")
    print("=" * 80)
    cm_optimal = confusion_matrix(y_test, y_pred_optimal)
    print("\n혼동 행렬:")
    print(cm_optimal)
    print(f"\n실제 하락 중 하락 예측: {cm_optimal[0,0]:,} / {cm_optimal[0].sum():,} ({cm_optimal[0,0]/cm_optimal[0].sum()*100:.1f}%)")
    print(f"실제 상승 중 상승 예측: {cm_optimal[1,1]:,} / {cm_optimal[1].sum():,} ({cm_optimal[1,1]/cm_optimal[1].sum()*100:.1f}%)")
    
    print("\n분류 성능:")
    print(classification_report(y_test, y_pred_optimal, target_names=['하락(0)', '상승(1)']))
    
    # === Feature Importance ===
    feature_importance = pd.DataFrame({
        'feature': feature_cols,
        'importance': rf_model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    print("\n" + "=" * 80)
    print("Feature Importance (Random Forest)")
    print("=" * 80)
    print(feature_importance)
    
    # 비교 요약
    print("\n" + "=" * 80)
    print("📈 성능 비교 요약")
    print("=" * 80)
    
    from sklearn.metrics import precision_score, recall_score, f1_score
    
    # Threshold 0.5
    precision_05 = precision_score(y_test, y_pred_default, pos_label=1)
    recall_05 = recall_score(y_test, y_pred_default, pos_label=1)
    f1_05 = f1_score(y_test, y_pred_default, pos_label=1)
    
    # Threshold 0.4
    precision_04 = precision_score(y_test, y_pred_optimal, pos_label=1)
    recall_04 = recall_score(y_test, y_pred_optimal, pos_label=1)
    f1_04 = f1_score(y_test, y_pred_optimal, pos_label=1)
    
    comparison = pd.DataFrame({
        'Threshold': [0.5, 0.4],
        'Precision': [precision_05, precision_04],
        'Recall': [recall_05, recall_04],
        'F1-Score': [f1_05, f1_04],
        'Predicted_Positive': [y_pred_default.sum(), y_pred_optimal.sum()]
    })
    
    print(comparison.to_string(index=False))
    print(f"\n✅ 추천: Threshold 0.4 사용")
    print(f"   - Recall {recall_04*100:.1f}%로 상승의 대부분 포착")
    print(f"   - F1-Score {f1_04:.3f}로 균형 최적")
    
else:
    print("\n⚠ 데이터가 부족하여 모델 학습을 건너뜁니다.")
    feature_importance = None
    rf_model = None
    y_proba = None
    y_pred_optimal = None

In [ ]:
# Feature Importance 시각화
if feature_importance is not None:
    plt.figure(figsize=(12, 8))
    plt.barh(range(len(feature_importance)), feature_importance['importance'], 
             alpha=0.7, edgecolor='black')
    plt.yticks(range(len(feature_importance)), feature_importance['feature'])
    plt.xlabel('Importance')
    plt.title('Feature Importance (Random Forest)', fontsize=14)
    plt.grid(True, alpha=0.3, axis='x')
    plt.tight_layout()
    plt.show()
    
    # 상위 10개 Feature
    print("\n상위 10개 중요한 Features:")
    for idx, row in feature_importance.head(10).iterrows():
        print(f"  {row['feature']:25s}: {row['importance']:.4f}")

# ROC 곡선 및 Precision-Recall 곡선
if rf_model is not None:
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # 1. ROC 곡선
    fpr, tpr, thresholds_roc = roc_curve(y_test, y_proba)
    roc_auc = auc(fpr, tpr)
    
    axes[0].plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.3f})')
    axes[0].plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random Baseline')
    axes[0].set_xlim([0.0, 1.0])
    axes[0].set_ylim([0.0, 1.05])
    axes[0].set_xlabel('False Positive Rate')
    axes[0].set_ylabel('True Positive Rate')
    axes[0].set_title('ROC Curve')
    axes[0].legend(loc="lower right")
    axes[0].grid(True, alpha=0.3)
    
    # 2. Precision-Recall 곡선
    precision, recall, thresholds_pr = precision_recall_curve(y_test, y_proba)
    
    axes[1].plot(recall, precision, color='blue', lw=2)
    axes[1].axhline(y=(y_test==1).sum()/len(y_test), color='red', linestyle='--', 
                    label=f'Baseline ({(y_test==1).sum()/len(y_test):.3f})')
    axes[1].set_xlim([0.0, 1.0])
    axes[1].set_ylim([0.0, 1.05])
    axes[1].set_xlabel('Recall')
    axes[1].set_ylabel('Precision')
    axes[1].set_title('Precision-Recall Curve')
    axes[1].legend(loc="upper right")
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print(f"\nROC AUC Score: {roc_auc:.3f}")

In [ ]:
# Threshold 조정 실험
if rf_model is not None:
    print("=" * 80)
    print("Threshold 조정 실험")
    print("=" * 80)
    print("\n다양한 Threshold에서의 성능 비교:")
    print(f"{'Threshold':<12} {'Precision':<12} {'Recall':<12} {'F1-Score':<12} {'예측 상승 수':<12}")
    print("-" * 80)
    
    threshold_results = []
    for threshold in [0.2, 0.3, 0.4, 0.5, 0.6, 0.7]:
        y_pred_thresh = (y_proba >= threshold).astype(int)
        
        # 상승(1) 클래스에 대한 지표만 계산
        from sklearn.metrics import precision_score, recall_score, f1_score
        
        precision = precision_score(y_test, y_pred_thresh, pos_label=1, zero_division=0)
        recall = recall_score(y_test, y_pred_thresh, pos_label=1)
        f1 = f1_score(y_test, y_pred_thresh, pos_label=1, zero_division=0)
        predicted_positive = y_pred_thresh.sum()
        
        threshold_results.append({
            'threshold': threshold,
            'precision': precision,
            'recall': recall,
            'f1': f1,
            'predicted_positive': predicted_positive
        })
        
        print(f"{threshold:<12.1f} {precision:<12.3f} {recall:<12.3f} {f1:<12.3f} {predicted_positive:<12,}")
    
    # 시각화
    df_thresh = pd.DataFrame(threshold_results)
    
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # 좌: Precision, Recall, F1-Score vs Threshold
    axes[0].plot(df_thresh['threshold'], df_thresh['precision'], 'o-', label='Precision', linewidth=2, markersize=8)
    axes[0].plot(df_thresh['threshold'], df_thresh['recall'], 's-', label='Recall', linewidth=2, markersize=8)
    axes[0].plot(df_thresh['threshold'], df_thresh['f1'], '^-', label='F1-Score', linewidth=2, markersize=8)
    axes[0].set_xlabel('Threshold')
    axes[0].set_ylabel('Score')
    axes[0].set_title('Performance Metrics vs Threshold')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    axes[0].set_xlim([0.15, 0.75])
    axes[0].set_ylim([0, 1])
    
    # 우: 예측된 상승 개수 vs Threshold
    axes[1].bar(df_thresh['threshold'], df_thresh['predicted_positive'], alpha=0.7, edgecolor='black')
    axes[1].axhline(y=(y_test==1).sum(), color='red', linestyle='--', linewidth=2, 
                    label=f'실제 상승 수: {(y_test==1).sum():,}')
    axes[1].set_xlabel('Threshold')
    axes[1].set_ylabel('Predicted Positive Count')
    axes[1].set_title('Predicted Positive Count vs Threshold')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # 최적 Threshold 찾기 (F1-Score 기준)
    best_idx = df_thresh['f1'].idxmax()
    best_threshold = df_thresh.loc[best_idx, 'threshold']
    best_f1 = df_thresh.loc[best_idx, 'f1']
    
    print(f"\n최적 Threshold (F1-Score 기준): {best_threshold:.1f}")
    print(f"  - F1-Score: {best_f1:.3f}")
    print(f"  - Precision: {df_thresh.loc[best_idx, 'precision']:.3f}")
    print(f"  - Recall: {df_thresh.loc[best_idx, 'recall']:.3f}")
    
    # 최적 Threshold로 재평가
    y_pred_optimal = (y_proba >= best_threshold).astype(int)
    cm_optimal = confusion_matrix(y_test, y_pred_optimal)
    
    print(f"\n최적 Threshold({best_threshold:.1f})에서의 혼동 행렬:")
    print(cm_optimal)
    print(f"\n실제 상승 중 상승 예측: {cm_optimal[1,1]:,} / {cm_optimal[1].sum():,} ({cm_optimal[1,1]/cm_optimal[1].sum()*100:.1f}%)")

## 6. 요약 및 인사이트

In [ ]:
print("=" * 80)
print("EDA 요약")
print("=" * 80)

print(f"\n1. 데이터 개요")
print(f"   - 전체 거래일: {len(df):,}")
print(f"   - 갭 상승 이벤트: {len(df_gap_up):,} ({len(df_gap_up)/len(df)*100:.1f}%)")
print(f"   - 기간: {df['date'].min().date()} ~ {df['date'].max().date()}")
print(f"   - 종목 수: {df['InfoCode'].nunique():,}")

print(f"\n2. 갭 상승 통계")
print(f"   - 평균 갭 상승률: {df_gap_up['gap_pct'].mean():.2f}%")
print(f"   - 중간 갭 상승률: {df_gap_up['gap_pct'].median():.2f}%")
print(f"   - 최대 갭 상승률: {df_gap_up['gap_pct'].max():.2f}%")

print(f"\n3. 타겟 분석")
print(f"   - 전체 승률: {df_gap_up['target_direction'].mean()*100:.2f}%")
print(f"   - 평균 수익률: {df_gap_up['target_return'].mean():.2f}%")
print(f"   - 상승 시 평균 수익률: {df_up['target_return'].mean():.2f}%")
print(f"   - 하락 시 평균 손실률: {df_down['target_return'].mean():.2f}%")

print(f"\n4. 주요 인사이트")
print(f"   - 갭 크기가 클수록 승률이 {'높습니다' if gap_size_stats['Win_Rate'].is_monotonic_increasing else '낮습니다'}")
print(f"   - 월요일 갭 상승 이벤트가 {'많습니다' if weekday_gaps['Gap_Events'].idxmax() == 'Monday' else '적습니다'}")

if feature_importance is not None:
    top_3_features = feature_importance.head(3)['feature'].tolist()
    print(f"   - 가장 중요한 Features: {', '.join(top_3_features)}")

print(f"\n5. 다음 단계")
print(f"   - Feature Engineering 추가 (이벤트/뉴스 데이터)")
print(f"   - 모델 학습 및 평가 (분류 + 회귀)")
print(f"   - 하이퍼파라미터 튜닝")
print(f"   - 백테스팅 시뮬레이션")

print("\n" + "=" * 80)

---

## 다음 노트북

- **03_feature_engineering.ipynb**: 추가 Feature 생성 (이벤트, 뉴스 등)
- **04_modeling_classifier.ipynb**: Model 1 (분류 모델) 학습
- **05_modeling_regressor.ipynb**: Model 2 & 3 (회귀 모델) 학습